In [18]:
from gurobipy import Model, GRB, quicksum

In [19]:
# Initialize the model
model = Model("Quayside_Apartment_Construction")

In [20]:
# Number of sites
N = 11

In [21]:
# Decision variables
x = model.addVars(N+1, vtype=GRB.BINARY, name="x")  # Style A decision
y = model.addVars(N+1, vtype=GRB.BINARY, name="y")  # Style B decision
z = model.addVars(N+1, vtype=GRB.INTEGER, name="z") # Number of units

In [22]:
# Objective function: Minimize the total cost
model.setObjective(quicksum(10000000 * n * x[n] + 10000000 * (11 - n) * y[n] + 50000 * z[n] for n in range(1, N+1)), GRB.MINIMIZE)

In [23]:
# Constraints

# Number of buildings between 4 and 9
model.addConstr(quicksum(x[n] + y[n] for n in range(1, N+1)) >= 4, "Min_Buildings")
model.addConstr(quicksum(x[n] + y[n] for n in range(1, N+1)) <= 9, "Max_Buildings")

# Units constraints for style A and B
for n in range(1, N+1):
    model.addConstr(z[n] >= 175 * x[n] + 210 * y[n], "Min_Units{}".format(n))
    model.addConstr(z[n] <= 245 * x[n] + 305 * y[n], "Max_Units{}".format(n))

# Total units must exceed 2400
model.addConstr(quicksum(z[n] for n in range(1, N+1)) >= 2400, "Total_Units")

# Site dependencies
model.addConstr(x[4] + y[4] <= x[7] + y[7], "Site_4_and_7_Dependency")

# Cross-style dependency
model.addConstr(x[5] <= y[6], "Style_Dependency_5_6")

# Ensure exclusivity of building styles at any site
for n in range(1, N+1):
    model.addConstr(x[n] + y[n] <= 1, "Style_Exclusivity_{}".format(n))

In [24]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 38 rows, 36 columns and 149 nonzeros
Model fingerprint: 0x140dde17
Variable types: 0 continuous, 36 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+04, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 4.600000e+08
Presolve removed 0 rows and 3 columns
Presolve time: 0.00s
Presolved: 38 rows, 33 columns, 149 nonzeros
Variable types: 0 continuous, 33 integer (22 binary)
Found heuristic solution: objective 4.000000e+08

Root relaxation: objective 3.028571e+08, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3.0286e

In [25]:
# Print solution
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for n in range(1, N+1):
        if x[n].X > 0.5:  # Check if style A is chosen
            print(f"Build a style A building at site {n} with {z[n].X} units.")
        elif y[n].X > 0.5:  # Check if style B is chosen
            print(f"Build a style B building at site {n} with {z[n].X} units.")
else:
    print("No optimal solution found.")

Optimal solution found:
Build a style A building at site 1 with 210.0 units.
Build a style A building at site 2 with 245.0 units.
Build a style A building at site 3 with 245.0 units.
Build a style A building at site 4 with 175.0 units.
Build a style B building at site 7 with 305.0 units.
Build a style B building at site 8 with 305.0 units.
Build a style B building at site 9 with 305.0 units.
Build a style B building at site 10 with 305.0 units.
Build a style B building at site 11 with 305.0 units.
